# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> Data Science 2: Advanced Topics in Data Science 
## Homework 3 - Bayesian Analysis



**Harvard University**<br/>
**Spring 2020**<br/>
**Instructors**: Mark Glickman, Pavlos Protopapas, & Chris Tanner 


<hr style="height:2pt">

### Homework 3 is due March 5th

In [1]:
#PLEASE RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment, please follow the instructions on Canvas.
- Please restart the kernel and run the entire notebook again before you submit.

<hr style="height:2pt">

### Please use the libraries below:

In [2]:
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
%matplotlib inline 
from datetime import datetime as dt
from scipy.special import expit

In [3]:
pm.__version__

'3.8'


<hr style="height:2pt">

### Contraceptive Usage by Bangladeshi Women

In the second part of this assignment, you are provided with data sets `dataset_2_train.csv` and `dataset_2_test.csv`containing details of contraceptive usage by 1934 Bangladeshi women. There are four attributes for each woman, along with a label indicating if she uses contraceptives. The attributes include
* district: identifying code for the district the woman lives in
* urban: type of region of residence
* living.children: number of living children
* age-mean: age of the woman (in years, centered around mean)

The women are grouped into 60 districts. The task is to build a classification model that can predict if a given woman uses contraceptives.


### Part A: Bayesian Logistic Regression

The first model we will fit to the contraceptives data is a varying-intercept logistic regression model, where the intercept varies by district.

Prior distribution:

$\beta_{0j} \sim N(\mu_0,\sigma_0^2),$ with $\mu_0 \sim N(0,10000)$ and $\frac{1}{\sigma^2_0} \sim \text{Gamma}(0.1,0.1)$.

$\beta_1 \sim N(0,10000)$, $\beta_2\sim N(0,10000)$, $\beta_3 \sim N(0,10000)$

Model for data:

$Y_{ij} \sim \text{Bernoulli}(p_{ij})$

$\text{logit}\:p_{ij} = \beta_{0j} + \beta_1 \times \text{urban} + \beta_2 \times \text{living.children} + \beta_3 \times \text{age-mean}$,

where $Y_{ij}$ is 1 if woman $i$ in district $j$ uses contraceptives, and 0 otherwise, and where $i \in \{1,...,N\}$ and $j \in \{1,...,J\}$. $N$ is the number of observations in the data, and $J$ is the number of districts. The above notation assumes $N(\mu, \sigma^2)$ is a Normal distribution with mean $\mu$ and variance $\sigma^2$.

##### **A1**

As a preprocessing step, map the district number 61 to the number 54 so that the districts are in order. Also, re-name district 60 to be district 0 so that the districts are numbered 0 through 59. **We use this numbering throughout the homework, and will grade using these district numbers**

In [4]:
# load data
bangla_train = pd.read_csv("../data/dataset_2_train.csv")
bangla_test = pd.read_csv("../data/dataset_2_test.csv")

display(bangla_train.shape, bangla_test.shape)
display(bangla_train.dtypes)
bangla_train.head()

(967, 5)

(967, 5)

district               int64
urban                  int64
living.children        int64
age_mean             float64
contraceptive_use      int64
dtype: object

,district,urban,living.children,age_mean,contraceptive_use
0,35,0,4,2.4400,0
1,22,0,2,-1.5599,1
2,29,0,2,-8.5599,1
3,5,0,3,-4.5599,1
4,34,1,4,8.4400,0


In [5]:
bangla_train.isnull().sum(), bangla_test.isnull().sum() # check for null values

(district             0
 urban                0
 living.children      0
 age_mean             0
 contraceptive_use    0
 dtype: int64,
 district             0
 urban                0
 living.children      0
 age_mean             0
 contraceptive_use    0
 dtype: int64)

In [6]:
display(np.sort(bangla_train.district.unique())) # make sure that 54 does not exist
len(bangla_train.district.unique()) # check for 60 unique values as mentioned

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 55, 56, 57, 58, 59, 60, 61])

60

In [7]:
display(np.sort(bangla_test.district.unique())) # repeat for test 
len(bangla_test.district.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 55, 56, 57, 58, 59, 60, 61])

60

In [8]:
# perform replace operation
bangla_train['district'].replace({61:54, 60:0}, inplace = True)
bangla_test.district.replace({61:54, 60:0}, inplace = True)

In [9]:
display(np.sort(bangla_train.district.unique())) # make sure that 54 does not exist
len(bangla_train.district.unique()) # check for 60 unique values as mentioned


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

60

In [10]:
display(np.sort(bangla_test.district.unique())) # repeat for test 
len(bangla_test.district.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

60

##### **A2**

The first model we will fit to the contraceptives data is a varying-intercept logistic regression model, where the intercept varies by district.

Prior distribution:

$\beta_{0j} \sim N(\mu_0,\sigma_0^2),$ with $\mu_0 \sim N(0,10000)$ and $\frac{1}{\sigma^2_0} \sim \text{Gamma}(0.1,0.1)$.

$\beta_1 \sim N(0,10000)$, $\beta_2\sim N(0,10000)$, $\beta_3 \sim N(0,10000)$

Model for data:

$Y_{ij} \sim \text{Bernoulli}(p_{ij})$

$\text{logit}\:p_{ij} = \beta_{0j} + \beta_1 \times \text{urban} + \beta_2 \times \text{living.children} + \beta_3 \times \text{age-mean}$,

where $Y_{ij}$ is 1 if woman $i$ in district $j$ uses contraceptives, and 0 otherwise, and where $i \in \{1,...,N\}$ and $j \in \{1,...,J\}$. $N$ is the number of observations in the data, and $J$ is the number of districts. The above notation assumes $N(\mu, \sigma^2)$ is a Normal distribution with mean $\mu$ and variance $\sigma^2$.

First, we'll verify that pymc3 can recover the hidden parameter values. To do so, we'll hard-code known values and simulate data from the model, then using that data we'll check if pymc3 can get back the parameters we hard-coded. If it does, we'll have hope that it can get the hidden parameter values that generated the real data.

Use the following hard-coded values:  
$\mu_0 = 2$

$\sigma^2_0 = 1$

$\beta_{0j} \sim N(\mu_0,\sigma_0^2)$ for all sixty districts

$\beta_1 = 4$

$\beta_2 = -3$

$\beta_3 = -2$

**Verify that these parameters give an overall average of 46 to 48 percent of subjects using contraceptives**

In [11]:
# specify all values
mu_0 = 2
sigma_0 = 1
beta_0 = np.random.normal(mu_0, sigma_0, 60)

beta_1 = 4
beta_2 = -3
beta_3 = -2

In [12]:
# delete before submission
# beta_0.reshape(-1,1)
# beta_0.shape
# type(beta_0)

In [13]:
# create beta_0 intercept
beta_0_intercept = beta_0[bangla_train['district']]

In [14]:
len(bangla_train)

967

In [15]:
np.random.seed(1234)
logit_p_i1 = beta_0_intercept + beta_1 * bangla_train['urban'] + beta_2 * bangla_train['living.children'] + beta_3 * bangla_train['age_mean']

logit_p_i1

0     -16.360218
1      -0.058412
2      12.382593
3       2.345093
4     -23.610884
         ...    
962     8.876822
963     4.774822
964     5.639582
965   -10.468054
966    17.401722
Length: 967, dtype: float64

In [16]:
p1 = expit(logit_p_i1)
p1[:5]

0    7.849603e-08
1    4.854011e-01
2    9.999958e-01
3    9.125434e-01
4    5.570873e-11
dtype: float64

In [18]:
contra_calc0 = np.random.binomial(1, p1)
value, counts = np.unique(contra_calc0, return_counts = True)

In [19]:
print("Percent of women using contraceptive:{:.2f}%".format(counts[1]*100/(np.sum(counts))))


Percent of women using contraceptive:47.67%


##### **A3**

Fit the varying-intercept model specified above to your simulated data. Reasonably good convergence may require adjustments to the number of tuning samples and the target acceptance rate as suggested in pymc3's warnings.

Note: It is wise to take advantage of a model's built-in `summary` method to inspect it after fitting

In [20]:
with pm.Model() as population_model1:
    
    np.random.seed(1234)
    tau_beta0 = pm.Gamma("tau_gamma", alpha = 0.1, beta = 0.1)
    mu_beta0 = pm.Normal("mu_beta0", mu = 0, tau = 10000)
    beta0 = pm.Normal('beta0', mu = mu_beta0, tau = tau_beta0, shape = len(bangla_train.district.unique()))
    
    beta1 = pm.Normal('beta1', mu = 0, tau = 10000)
    beta2 = pm.Normal('beta2', mu = 0, tau = 10000)
    beta3 = pm.Normal('beta3', mu = 0, tau = 10000)

    beta0_intercept = beta0[bangla_train['district']]
    
    logit_p_i1 = pm.Deterministic('p_i1', pm.math.invlogit(beta0_intercept + beta1*bangla_train['urban'] + beta2*bangla_train['living.children'] + beta3*bangla_train['age_mean']))
    
    contra_calc1 = pm.Binomial('obs_num_women', n = 1, p = logit_p_i1, observed = contra_calc0)
    
    trace = pm.sample(10000, tune = 5000, target_accept = 0.9)
    

/opt/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/opt/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta3, beta2, beta1, beta0, mu_beta0, tau_gamma]
/opt/anaconda3/lib/python3.7/site-packages/

In [21]:
pm.summary(trace, var_names = ['mu_beta0','tau_gamma','beta1','beta2','beta3'])


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu_beta0,-0.002,0.010,-0.021,0.017,0.000,0.000,33796.0,8699.0,33796.0,14018.0,1.0
tau_gamma,19.937,10.752,4.721,39.749,0.243,0.172,1965.0,1965.0,1946.0,3775.0,1.0
beta1,0.001,0.010,-0.018,0.019,0.000,0.000,29760.0,7747.0,29768.0,14106.0,1.0
beta2,-0.028,0.009,-0.046,-0.010,0.000,0.000,30156.0,25403.0,30116.0,12829.0,1.0
beta3,-0.136,0.007,-0.149,-0.123,0.000,0.000,41259.0,40977.0,41187.0,14824.0,1.0


##### **A4**

Plot the trace plots of the MCMC sampler for the parameters $\mu_0, \sigma_0, \beta_1, \beta_2, \beta_3$. Based on these and the R-hat values, does it look like the samplers converged?

In [ ]:
pm.traceplot(trace, ['mu_beta0','tau_gamma','beta1','beta2','beta3']);

In [ ]:
pm.plots.plot_posterior(trace,                          var_names=['$\\beta_0$'], kind='hist',                          coords={'$\\beta_0$_dim_0':np.arange(9,39,10)})

In [ ]:
#your code here


In [ ]:
#your code here


your response here

##### **A5**

Plot histograms of the posterior distributions for the parameters $\beta_{0,9}$, $\beta_{0,19}$, $\beta_{0,29}$, ..., $\beta_{0,59}$. Are the actual parameters that you generated contained within these posterior distributions? 

In [ ]:
#your code here


your response here


##### **A6**

We now fit our model to the actual data. Fit the varying-intercept model to the real training data.

In [ ]:
with pm.Model() as population_model2:
    
    tau_beta0 = 1./pm.Gamma("tau_gamma", alpha = 0.1, beta = 0.1)
    mu_beta0 = pm.Normal("mu_beta0", mu = 0, tau = 10000)
    beta0 = pm.Normal('beta0', mu = mu_beta0, tau = tau_beta0, shape = len(bangla_train.district.unique()))
    
    beta1 = pm.Normal('beta1', mu = 0, tau = 10000)
    beta2 = pm.Normal('beta2', mu = 0, tau = 10000)
    beta3 = pm.Normal('beta3', mu = 0, tau = 10000)

    beta0_intercept = beta0[bangla_train['district']]
    
    logit_p_i1 = pm.Deterministic('p_i1', pm.math.invlogit(beta0_intercept + beta1*bangla_train['urban'] + beta2*bangla_train['living.children'] + beta3*bangla_train['age_mean']))
    
    contra_calc2 = pm.Binomial('obs_num_women', n = 1, p = logit_p_i1, observed = bangla_train['contraceptive_use'])
    
    trace = pm.sample(6000, tune = 3000, target_accept = 0.9)
    

In [ ]:
#your code here


##### **A7**

Check the convergence by examining the trace plots and R-hats, as you did with the simulated data. What do you observe?

In [ ]:
#your code here


your response here

##### **A8**

Based on the posterior means, which district has the highest base rate of contraceptive usage (independent of other factors like urban population)? Which district has the lowest?

In [ ]:
#your code here


your response here

##### **A9**

What are the posterior means of $\mu_0$ and $\sigma^2_0$? Do these values offer any evidence in support of or against the varying-intercept model, compared to a model with a single intercept value for all districts or a model with an intercept of 0 for all districts?

In [ ]:
#your code here


your response here

### Part B: Varying-Coefficients Model

The next model we will fit to the contraceptives data is a varying-coefficients logistic regression model, where the coefficients on living.children, age-mean, and urban vary by district.

$\beta_{0j} \sim N(\mu_0,\sigma_0^2),$ with $\mu_0 \sim N(0,10000)$ and $\frac{1}{\sigma_0^2} \sim \text{Gamma}(0.1,0.1)$.

$\beta_{1j} \sim N(0,\sigma_1^2)$, with $\frac{1}{\sigma^2_1} \sim \text{Gamma}(0.1,0.1)$

$\beta_{2j}\sim N(0,\sigma_2^2)$, with $\frac{1}{\sigma^2_2}\sim \text{Gamma}(0.1,0.1)$

$\beta_{3j} \sim N(0,\sigma_3^2)$, with $\frac{1}{\sigma^2_3} \sim \text{Gamma}(0.1,0.1)$

Model for data:

$Y_{ij} \sim \text{Bernoulli}(p_{ij})$

$\text{logit}\:p_{ij} = \beta_{0j} + \beta_{1j} \times \text{urban} + \beta_{2j} \times \text{living.children} + \beta_{3j} \times \text{age-mean}$,

##### **B1**

Fit the model to the real data. Plot posterior means for each of the coefficients to the predictors. What do you conclude from these graphs?

In [ ]:
#your code here


your response here

##### **B2**

Use all of the information you've gleaned thus far to build a final Bayesian logistic regression classifier on the train set. Then, use your model to make predictions on the test set. Report your model's classification percentage.

In [ ]:
#your code here


your response here